In [12]:
import json
import glob
import os
import pandas as pd
import numpy as np

## 1. Settings to check:

-> activate proj. pushdown, filter pushdown

- hyper / no-cf  invoked via `python3.9 runtuplex.py --no-cf`
- nohyper / no-cf invoked via `python3.9 runtuplex.py --no-cf --no-hyper`
- hyper / cf invoked via `python3.9 runtuplex.py`
- nohyper / cf `python3.9 runtuplex.py --no-hyper`

All of them should have the same number of output rows, input rows.

In [13]:
!ls flights_debug/

job_0000.json job_0001.json job_0002.json job_0003.json


In [31]:
path_map = {
    'flights_debug/job_0000.json':'hyper-nocf',
           'flights_debug/job_0001.json':'nohyper-nocf',
          'flights_debug/job_0002.json':'hyper-cf',
           'flights_debug/job_0003.json':'nohyper-cf'}

In [32]:
path = list(path_map.keys())[0]

In [33]:
for path in path_map.keys():
    with open(path, 'r') as fp:
        print(path_map[path])
        data = json.load(fp)
        print(data['input_paths_taken'])
        print(data['output_paths_taken'])
        x = np.array(list(data['input_paths_taken'].values()))
        print('total: {}'.format(x.sum()))
        print('---')

hyper-nocf
{'normal': 202449065, 'general': 238590, 'fallback': 0, 'unresolved': 0}
{'normal': 197327045, 'unresolved': 0}
total: 202687655
---
nohyper-nocf
{'normal': 202449065, 'general': 238590, 'fallback': 0, 'unresolved': 0}
{'normal': 175791035, 'unresolved': 0}
total: 202687655
---
hyper-cf
{'normal': 144522313, 'general': 57974043, 'fallback': 0, 'unresolved': 191299}
{'normal': 198926126, 'unresolved': 191299}
total: 202687655
---
nohyper-cf
{'normal': 202449065, 'general': 238590, 'fallback': 0, 'unresolved': 0}
{'normal': 175791035, 'unresolved': 0}
total: 202687655
---


In [28]:
data['output_paths_taken']

{'normal': 175791035, 'unresolved': 0}

In [22]:
test_path = '/Users/leonhards/projects/tuplex-public/tuplex/test/resources/hyperspecialization/flights/flights_on_time_performance_2021_11.csv.sample'

In [24]:
df = pd.read_csv(test_path)

In [26]:
df['NAS_DELAY']

0       29.0
1       22.0
2       31.0
3       23.0
4       20.0
        ... 
4994     NaN
4995     NaN
4996     NaN
4997     NaN
4998     NaN
Name: NAS_DELAY, Length: 4999, dtype: float64

Next test: write pipeline to (only) extract year and check in all modes.
    

=> #input rows should match #output rows. If not, debug and fix!

- number of input rows is: 202687655
- number of output rows should be the same!: 202687655

In [ ]:
[2022-12-04 13:54:36.257] [aws-lambda] [info] LAMBDA Stage output: 202687655 rows, 0 exceptions
[2022-12-04 13:54:36.257] [aws-lambda] [info] LAMBDA paths input rows took: normal: 202687655 general: 0 interpreter: 0

In [36]:
# for nas delay, hyper fails with: 201224513 vs. 202687655 -> incorrect number of rows.

In [37]:
# nohyper as well: 180051706

In [38]:
#nohyper/nocf as well: 180051706

In [39]:
prob. the null check is off. Need to investigate, fix this.

SyntaxError: invalid syntax (2346267872.py, line 1)

## --- a couple bugs fixed --- yet excepitons in hyper mode with cf.

In [40]:
path = 'flights_debug/job_0044.json'

In [41]:
    with open(path, 'r') as fp:
        data = json.load(fp)
        print(data['input_paths_taken'])
        print(data['output_paths_taken'])
        x = np.array(list(data['input_paths_taken'].values()))
        print('total: {}'.format(x.sum()))
        print('---')

{'normal': 140952083, 'general': 61544273, 'fallback': 0, 'unresolved': 191299}
{'normal': 202496356, 'unresolved': 191299}
total: 202687655
---


In [42]:
data.keys()

dict_keys(['stageStartTimestamp', 'stageEndTimestamp', 'hyper_mode', 'cost', 'input_paths_taken', 'output_paths_taken', 'tasks', 'requests'])

In [46]:
list(filter(lambda x: x['input_paths_taken']['unresolved'] != 0, data['requests']))[0]

{'requestId': '6beb83d5-8519-4e95-88ef-4c45143d39c3',
 'containerId': '8f68e195-8cc5-474c-855e-41b9f8854357',
 'durationInMs': 7279.16,
 'billedDurationInMs': 7280,
 'memorySizeInMb': 10000,
 'maxMemoryUsedInMb': 491,
 'returnCode': 0,
 'errorMessage': '',
 'input_paths_taken': {'normal': 8390,
  'general': 418457,
  'fallback': 0,
  'unresolved': 244},
 'output_paths_taken': {'normal': 426847, 'unresolved': 244},
 't_hyper': 0.609789,
 't_compile': 0.0814773,
 't_fast': 5.17275,
 't_slow': 0.949678,
 'tsRequestStart': 1670191260099440813,
 'tsRequestEnd': 1670191267571687332}

In [50]:
list(filter(lambda x: x['container']['requestId'] == '6beb83d5-8519-4e95-88ef-4c45143d39c3', data['tasks']))

[{'container': {'reused': True,
   'requestId': '6beb83d5-8519-4e95-88ef-4c45143d39c3',
   'uuid': '8f68e195-8cc5-474c-855e-41b9f8854357',
   'msRemaining': 592725,
   'requestsServed': 31,
   'startTimestamp': 1670188942828895966,
   'deadlineTimestamp': 1670191860279000000},
  'invoked_containers': [],
  'log': "[2022-12-04 22:00:33.285] [Lambda worker] [info] performing global initialization (Worker App)\n[2022-12-04 22:00:33.290] [Lambda worker] [info] Invoking WorkerApp fallback\n[2022-12-04 22:00:33.292] [Lambda worker] [info] Found 1 input URIs to process\n[2022-12-04 22:00:33.292] [Lambda worker] [info] *** hyperspecialization active ***\n[2022-12-04 22:00:33.292] [Lambda worker] [info] -- specializing to s3://tuplex-public/data/flights_all/flights_on_time_performance_1995_11.csv:0-189889530\n[2022-12-04 22:00:33.292] [hyper specializer] [info] specializing code to file s3://tuplex-public/data/flights_all/flights_on_time_performance_1995_11.csv:0-189889530\n[2022-12-04 22:00:33

In [53]:
print(data['tasks'][100]['log'])

[2022-12-04 22:00:33.169] [Lambda worker] [info] performing global initialization (Worker App)
[2022-12-04 22:00:33.173] [Lambda worker] [info] Invoking WorkerApp fallback
[2022-12-04 22:00:33.175] [Lambda worker] [info] Found 1 input URIs to process
[2022-12-04 22:00:33.175] [Lambda worker] [info] *** hyperspecialization active ***
[2022-12-04 22:00:33.175] [Lambda worker] [info] -- specializing to s3://tuplex-public/data/flights_all/flights_on_time_performance_1994_08.csv:0-187864859
[2022-12-04 22:00:33.175] [hyper specializer] [info] specializing code to file s3://tuplex-public/data/flights_all/flights_on_time_performance_1994_08.csv:0-187864859
[2022-12-04 22:00:33.176] [hyper specializer] [info] Decompressed Code context from 9.38 KB to 88.28 KB
[2022-12-04 22:00:33.210] [hyper specializer] [info] Deserialization of Code context took 0.033519s
[2022-12-04 22:00:33.210] [hyper specializer] [info] Total Stage Decode took 0.034337s
[2022-12-04 22:00:33.210] [hyper specializer] [info